In [1]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.stats import norm
from scipy import stats
import seaborn as sns
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d

In [2]:
def divide_data_by_flow_direction(data):
    '''Dividing raw data from one fish into 2 dataframes with either left or right OMR flow'''
    data.columns = ['X_coord', 'Y_coord', 'heading_direction', \
                'cumulative_direction','beat_freq', 'beat_amp', \
                'tail_move?', 'timestamp', 'contrast_level', 'flow_direction']
    right = pd.DataFrame(data[data.flow_direction == 1])
    left = pd.DataFrame(data[data.flow_direction == 2])
    zero = pd.DataFrame(data[data.flow_direction == 0])

    return right, left, zero

def divide_data_by_contrast(data):
    '''Dividing raw data from one fish and one flow direction into contrast levels'''
    C_0 = pd.DataFrame(data[data.contrast_level == 0])
    C_01 = pd.DataFrame(data[data.contrast_level == 0.01])
    C_1 = pd.DataFrame(data[data.contrast_level == 0.1])
    C_2 = pd.DataFrame(data[data.contrast_level == 0.2])
    C_3 = pd.DataFrame(data[data.contrast_level == 0.3])
    C_5 = pd.DataFrame(data[data.contrast_level == 0.5])
    C_7 = pd.DataFrame(data[data.contrast_level == 0.7])
    C_10 = pd.DataFrame(data[data.contrast_level == 1])
    return C_0, C_01, C_1, C_2, C_3, C_5, C_7, C_10

def omr_preprocess(data):
    # remove timestamp and extras
    # keep x, y, heading, cumulative, timestamp
    data.columns = ['X_coord', 'Y_coord', 'heading_direction', \
                'cumulative_direction','beat_freq', 'beat_amp', \
                'tail_move?', 'timestamp', 'contrast_level', 'flow_direction']
    new = data.drop(columns = ['beat_freq', 'beat_amp','tail_move?', 'contrast_level', 'flow_direction'])


    # resetting index
    new = pd.DataFrame(new)
    new = new.set_index('timestamp').reset_index()


    # remove time points where there was an angle change of more than pi from one frame to another
    for i, row in new.iterrows():
        if i+1 == len(new):
            break
            # modify to add the exclusion zone
        if np.abs(new.at[i+1,'cumulative_direction']-new.at[i,'cumulative_direction']) >= 2.5:
            new.at[i+1, 'cumulative_direction'] = new.at[i,'cumulative_direction']


    # interpolating and normalising data to a fixed set of points
    interp = pd.DataFrame(columns=['timestamp','X_coord','Y_coord','heading_direction','cumulative_direction'])
    for column in new.columns:
        x = np.arange(0,len(new))
        y = new[column]
        f = interpolate.interp1d(x,y)

        x_new = np.arange(0,3000,1)
        y_new = f(x_new)
        interp[column] = y_new


    # setting first cumulative_angle to zero and ajdusting all others
    interp.iloc[:,4] -= interp.iloc[0,4]


    # calculating distance traveled between each timeframe
    # distance = sqrt((x2-x1)**2 + (y2-y1)**2)
    interp['distance_pts'] = 0
    for row in range(1,len(interp),1):
        distance = np.sqrt((interp['X_coord'][row]-interp['X_coord'][row-1])**2\
                            +(interp['Y_coord'][row]-interp['Y_coord'][row-1])**2)
        interp.iloc[row,5] = distance


    # cleaning the timestamps
    interp.insert(0, 'new_timestamp', range(1, 1 + len(interp)))
    interp = interp.drop(columns=['timestamp']).rename(columns={'new_timestamp':'timestamp'})
    interp['timestamp'] = interp['timestamp']/100

    return np.array(interp)

def combine_fish_data(*args):
    '''Combines preprocessed data from all fish into a 3D numpy array'''
    combined_fish = np.stack((args),axis=0)
    return combined_fish

In [3]:
for e in range(1,19):
    exec(f'e{e} = pd.read_csv("../raw_data/PM/E2/xy_hc_fai_tstp_E2_{e}_PM.csv")')
    
for e in range(1,19):
    exec(f'Re{e},Le{e},Ze{e} = divide_data_by_flow_direction(e{e})')

for flow in ['R','L']:
    for e in range(1,19):
        exec(f'{flow}e{e}C_0, {flow}e{e}C_1, {flow}e{e}C_10, {flow}e{e}C_20, {flow}e{e}C_30, \
            {flow}e{e}C_50, {flow}e{e}C_70, {flow}e{e}C_100 = divide_data_by_contrast({flow}e{e})')

for flow in ['R','L']:
    for contrast in ['1','10','20','30','50','70','100']:
        for e in range(1,19):
            exec(f'{flow}e{e}C_{contrast} = omr_preprocess({flow}e{e}C_{contrast})')

In [5]:
for me in range(1,23):
    exec(f'me{me} = pd.read_csv("../raw_data/PM/Met/xy_hc_fai_tstp_Me_{me}_PM.csv")')
    
for me in range(1,23):
    exec(f'Rme{me},Lme{me},Zme{me} = divide_data_by_flow_direction(me{me})')

for flow in ['R','L']:
    for me in range(1,23):
        exec(f'{flow}me{me}C_0, {flow}me{me}C_1, {flow}me{me}C_10, {flow}me{me}C_20, {flow}me{me}C_30, \
            {flow}me{me}C_50, {flow}me{me}C_70, {flow}me{me}C_100 = divide_data_by_contrast({flow}me{me})')

for flow in ['R','L']:
    for contrast in ['1','10','20','30','50','70','100']:
        for me in range(1,23):
            exec(f'{flow}me{me}C_{contrast} = omr_preprocess({flow}me{me}C_{contrast})')

In [ ]:
comb_LeC1 = combine_fish_data(Le1C_1,Le2C_1,Le3C_1,Le4C_1,Le5C_1,Le6C_1,Le8C_1,\
                            Le10C_1,Le11C_1,Le12C_1,Le13C_1,Le14C_1,Le15C_1,Le16C_1,Le17C_1,Le18C_1)
comb_LeC10 = combine_fish_data(Le1C_10,Le2C_10,Le3C_10,Le4C_10,Le5C_10,Le6C_10,Le7C_10,Le8C_10,\
                            Le11C_10,Le12C_10,Le13C_10,Le14C_10,Le15C_10,Le16C_10,Le17C_10,Le18C_10)
comb_LeC20 = combine_fish_data(Le1C_20,Le2C_20,Le3C_20,Le4C_20,Le5C_20,Le6C_20,Le7C_20,Le8C_20,Le9C_20,\
                            Le10C_20,Le11C_20,Le12C_20,Le13C_20,Le14C_20,Le15C_20,Le16C_20,Le17C_20,Le18C_20)
comb_LeC30 = combine_fish_data(Le1C_30,Le2C_30,Le3C_30,Le4C_30,Le5C_30,Le6C_30,Le7C_30,Le8C_30,\
                            Le10C_30,Le11C_30,Le12C_30,Le13C_30,Le14C_30,Le15C_30,Le16C_30,Le17C_30,Le18C_30)
comb_LeC50 = combine_fish_data(Le1C_50,Le2C_50,Le3C_50,Le4C_50,Le5C_50,Le6C_50,Le7C_50,Le8C_50,Le9C_50,\
                            Le10C_50,Le11C_50,Le12C_50,Le13C_50,Le14C_50,Le15C_50,Le17C_50,Le18C_50)
comb_LeC70 = combine_fish_data(Le1C_70,Le2C_70,Le3C_70,Le4C_70,Le5C_70,Le6C_70,Le7C_70,Le8C_70,\
                            Le10C_70,Le11C_70,Le12C_70,Le13C_70,Le14C_70,Le15C_70,Le16C_70,Le17C_70,Le18C_70)
comb_LeC100 = combine_fish_data(Le1C_100,Le2C_100,Le3C_100,Le4C_100,Le5C_100,Le6C_100,Le7C_100,Le8C_100,Le9C_100,\
                            Le10C_100,Le13C_100,Le14C_100,Le16C_100,Le17C_100,Le18C_100)

comb_ReC1 = combine_fish_data(Re1C_1,Re2C_1,Re3C_1,Re5C_1,Re6C_1,Re8C_1,Re9C_1,\
                            Re10C_1,Re11C_1,Re12C_1,Re13C_1,Re14C_1,Re15C_1,Re17C_1,Re18C_1)
comb_ReC10 = combine_fish_data(Re1C_10,Re2C_10,Re3C_10,Re4C_10,Re5C_10,Re6C_10,Re7C_10,Re8C_10,Re9C_10,\
                            Re10C_10,Re11C_10,Re12C_10,Re13C_10,Re14C_10,Re15C_10,Re16C_10,Re17C_10,Re18C_10)
comb_ReC20 = combine_fish_data(Re1C_20,Re2C_20,Re3C_20,Re4C_20,Re5C_20,Re6C_20,Re7C_20,Re8C_20,\
                            Re10C_20,Re11C_20,Re12C_20,Re13C_20,Re14C_20,Re15C_20,Re16C_20,Re17C_20,Re18C_20)
comb_ReC30 = combine_fish_data(Re1C_30,Re2C_30,Re3C_30,Re4C_30,Re5C_30,Re6C_30,Re7C_30,Re8C_30,\
                            Re10C_30,Re11C_30,Re12C_30,Re13C_30,Re14C_30,Re15C_30,Re17C_30,Re18C_30)
comb_ReC50 = combine_fish_data(Re1C_50,Re2C_50,Re3C_50,Re4C_50,Re5C_50,Re6C_50,Re7C_50,Re8C_50,\
                            Re10C_50,Re11C_50,Re12C_50,Re13C_50,Re14C_50,Re16C_50,Re17C_50,Re18C_50)
comb_ReC70 = combine_fish_data(Re1C_70,Re2C_70,Re3C_70,Re4C_70,Re5C_70,Re6C_70,Re7C_70,Re8C_70,Re9C_70,\
                            Re10C_70,Re11C_70,Re12C_70,Re13C_70,Re14C_70,Re15C_70,Re16C_70,Re17C_70,Re18C_70)
comb_ReC100 = combine_fish_data(Re1C_100,Re2C_100,Re3C_100,Re4C_100,Re5C_100,Re6C_100,Re7C_100,Re8C_100,Re9C_100,\
                            Re10C_100,Re11C_100,Re12C_100,Re13C_100,Re14C_100,Re15C_100,Re16C_100,Re17C_100,Re18C_100)

In [ ]:
comb_LmeC1 = combine_fish_data(Lme1C_1,Lme2C_1,Lme3C_1,Lme4C_1,Lme5C_1,Lme6C_1,Lme7C_1,Lme8C_1,Lme9C_1,\
                                Lme10C_1,Lme11C_1,Lme12C_1,Lme13C_1,Lme14C_1,Lme15C_1,Lme16C_1,Lme17C_1,Lme18C_1,\
                                Lme19C_1,Lme20C_1,Lme21C_1,Lme22C_1)
comb_LmeC10 = combine_fish_data(Lme1C_10,Lme2C_10,Lme3C_10,Lme4C_10,Lme5C_10,Lme6C_10,Lme7C_10,Lme9C_10,\
                                 Lme10C_10,Lme11C_10,Lme12C_10,Lme13C_10,Lme14C_10,Lme15C_10,Lme17C_10,Lme18C_10,\
                                Lme19C_10,Lme20C_10,Lme21C_10,Lme22C_10)
comb_LmeC20 = combine_fish_data(Lme1C_20,Lme2C_20,Lme3C_20,Lme4C_20,Lme5C_20,Lme6C_20,Lme7C_20,Lme8C_20,Lme9C_20,\
                                 Lme10C_20,Lme11C_20,Lme12C_20,Lme13C_20,Lme14C_20,Lme15C_20,Lme16C_20,Lme17C_20,\
                                 Lme19C_20,Lme20C_20,Lme21C_20,Lme22C_20)
comb_LmeC30 = combine_fish_data(Lme1C_30,Lme2C_30,Lme3C_30,Lme4C_30,Lme5C_30,Lme6C_30,Lme7C_30,Lme8C_30,Lme9C_30,\
                                 Lme10C_30,Lme11C_30,Lme12C_30,Lme13C_30,Lme14C_30,Lme15C_30,Lme17C_30,\
                                Lme18C_30,Lme19C_30,Lme20C_30,Lme21C_30,Lme22C_30)
comb_LmeC50 = combine_fish_data(Lme1C_50,Lme2C_50,Lme3C_50,Lme4C_50,Lme5C_50,Lme6C_50,Lme7C_50,Lme8C_50,Lme9C_50,\
                                 Lme10C_50,Lme11C_50,Lme12C_50,Lme13C_50,Lme14C_50,Lme15C_50,Lme16C_50,Lme17C_50,\
                                Lme18C_50,Lme19C_50,Lme20C_50,Lme21C_50,Lme22C_50)
comb_LmeC70 = combine_fish_data(Lme2C_70,Lme3C_70,Lme4C_70,Lme5C_70,Lme6C_70,Lme7C_70,Lme8C_70,Lme9C_70,\
                                 Lme10C_70,Lme11C_70,Lme12C_70,Lme13C_70,Lme14C_70,Lme15C_70,Lme16C_70,Lme17C_70,\
                                Lme18C_70,Lme19C_70,Lme20C_70,Lme21C_70,Lme22C_70)
comb_LmeC100 = combine_fish_data(Lme2C_100,Lme3C_100,Lme4C_100,Lme5C_100,Lme6C_100,Lme7C_100,Lme8C_100,\
                                  Lme9C_100,Lme10C_100,Lme11C_100,Lme12C_100,Lme13C_100,Lme14C_100,Lme15C_100,\
                                  Lme16C_100,Lme17C_100,Lme18C_100,Lme19C_100,Lme20C_100,Lme21C_100,Lme22C_100)

comb_RmeC1 = combine_fish_data(Rme1C_1,Rme2C_1,Rme3C_1,Rme4C_1,Rme5C_1,Rme6C_1,Rme7C_1,Rme8C_1,Rme9C_1,\
                                 Rme10C_1,Rme11C_1,Rme12C_1,Rme13C_1,Rme14C_1,Rme15C_1,Rme16C_1,Rme17C_1,\
                                 Rme18C_1,Rme19C_1,Rme20C_1,Rme21C_1,Rme22C_1)
comb_RmeC10 = combine_fish_data(Rme1C_10,Rme2C_10,Rme3C_10,Rme4C_10,Rme5C_10,Rme6C_10,Rme7C_10,Rme8C_10,Rme9C_10,\
                                 Rme10C_10,Rme11C_10,Rme12C_10,Rme13C_10,Rme14C_10,Rme15C_10,Rme16C_10,Rme17C_10,\
                                 Rme18C_10,Rme20C_10,Rme21C_10,Rme22C_10)
comb_RmeC20 = combine_fish_data(Rme1C_20,Rme2C_20,Rme3C_20,Rme4C_20,Rme5C_20,Rme6C_20,Rme7C_20,Rme8C_20,Rme9C_20,\
                                 Rme10C_20,Rme11C_20,Rme12C_20,Rme14C_20,Rme15C_20,Rme16C_20,Rme17C_20,\
                                 Rme18C_20,Rme19C_20,Rme20C_20,Rme21C_20,Rme22C_20)
comb_RmeC30 = combine_fish_data(Rme1C_30,Rme2C_30,Rme3C_30,Rme4C_30,Rme5C_30,Rme6C_30,Rme7C_30,Rme8C_30,Rme9C_30,\
                                 Rme10C_30,Rme11C_30,Rme12C_30,Rme13C_30,Rme14C_30,Rme15C_30,Rme16C_30,Rme17C_30,\
                                 Rme18C_30,Rme19C_30,Rme20C_30,Rme21C_30,Rme22C_30)
comb_RmeC50 = combine_fish_data(Rme1C_50,Rme2C_50,Rme3C_50,Rme4C_50,Rme5C_50,Rme6C_50,Rme7C_50,Rme8C_50,Rme9C_50,\
                                 Rme10C_50,Rme11C_50,Rme12C_50,Rme13C_50,Rme14C_50,Rme15C_50,Rme16C_50,Rme17C_50,\
                                 Rme18C_50,Rme19C_50,Rme20C_50,Rme21C_50,Rme22C_50)
comb_RmeC70 = combine_fish_data(Rme1C_70,Rme2C_70,Rme3C_70,Rme4C_70,Rme5C_70,Rme6C_70,Rme7C_70,Rme8C_70,Rme9C_70,\
                                 Rme10C_70,Rme11C_70,Rme12C_70,Rme13C_70,Rme14C_70,Rme15C_70,Rme16C_70,Rme17C_70,\
                                 Rme18C_70,Rme19C_70,Rme20C_70,Rme21C_70,Rme22C_70)
comb_RmeC100 = combine_fish_data(Rme1C_100,Rme2C_100,Rme3C_100,Rme4C_100,Rme5C_100,Rme6C_100,Rme7C_100,Rme8C_100,\
                                  Rme9C_100,Rme10C_100,Rme11C_100,Rme12C_100,Rme13C_100,Rme14C_100,Rme15C_100,\
                                  Rme16C_100,Rme17C_100,Rme18C_100,Rme19C_100,Rme20C_100,Rme21C_100,Rme22C_100) 

In [ ]:
comb_LeC1[:,:,4] = abs(comb_LeC1[:,:,4])
comb_LeC10[:,:,4] = abs(comb_LeC10[:,:,4])
comb_LeC20[:,:,4] = abs(comb_LeC20[:,:,4])
comb_LeC30[:,:,4] = abs(comb_LeC30[:,:,4])
comb_LeC50[:,:,4] = abs(comb_LeC50[:,:,4])
comb_LeC70[:,:,4] = abs(comb_LeC70[:,:,4])
comb_LeC100[:,:,4] = abs(comb_LeC100[:,:,4])

In [ ]:
comb_LmeC1[:,:,4] = abs(comb_LmeC1[:,:,4])
comb_LmeC10[:,:,4] = abs(comb_LmeC10[:,:,4])
comb_LmeC20[:,:,4] = abs(comb_LmeC20[:,:,4])
comb_LmeC30[:,:,4] = abs(comb_LmeC30[:,:,4])
comb_LmeC50[:,:,4] = abs(comb_LmeC50[:,:,4])
comb_LmeC70[:,:,4] = abs(comb_LmeC70[:,:,4])
comb_LmeC100[:,:,4] = abs(comb_LmeC100[:,:,4])

In [ ]:
eC1 = np.vstack((comb_LeC1,comb_ReC1))
eC10 = np.vstack((comb_LeC10,comb_ReC10))
eC20 = np.vstack((comb_LeC20,comb_ReC20))
eC30 = np.vstack((comb_LeC30,comb_ReC30))
eC50 = np.vstack((comb_LeC50,comb_ReC50))
eC70 = np.vstack((comb_LeC70,comb_ReC70))
eC100 = np.vstack((comb_LeC100,comb_ReC100))

In [ ]:
meC1 = np.vstack((comb_LmeC1,comb_RmeC1))
meC10 = np.vstack((comb_LmeC10,comb_RmeC10))
meC20 = np.vstack((comb_LmeC20,comb_RmeC20))
meC30 = np.vstack((comb_LmeC30,comb_RmeC30))
meC50 = np.vstack((comb_LmeC50,comb_RmeC50))
meC70 = np.vstack((comb_LmeC70,comb_RmeC70))
meC100 = np.vstack((comb_LmeC100,comb_RmeC100))

In [ ]:
eC1_cum = eC1[:,-1,4]
eC10_cum = eC10[:,-1,4]
eC20_cum = eC20[:,-1,4]
eC30_cum = eC30[:,-1,4]
eC50_cum = eC50[:,-1,4]
eC70_cum = eC70[:,-1,4]
eC100_cum = eC100[:,-1,4]

In [ ]:
eC1_sem = stats.sem(eC1[:,-1,4])
eC10_sem = stats.sem(eC10[:,-1,4])
eC20_sem = stats.sem(eC20[:,-1,4])
eC30_sem = stats.sem(eC30[:,-1,4])
eC50_sem = stats.sem(eC50[:,-1,4])
eC70_sem = stats.sem(eC70[:,-1,4])
eC100_sem = stats.sem(eC100[:,-1,4])

In [ ]:
meC1_cum = meC1[:,-1,4]
meC10_cum = meC10[:,-1,4]
meC20_cum = meC20[:,-1,4]
meC30_cum = meC30[:,-1,4]
meC50_cum = meC50[:,-1,4]
meC70_cum = meC70[:,-1,4]
meC100_cum = meC100[:,-1,4]

In [ ]:
meC1_sem = stats.sem(meC1[:,-1,4])
meC10_sem = stats.sem(meC10[:,-1,4])
meC20_sem = stats.sem(meC20[:,-1,4])
meC30_sem = stats.sem(meC30[:,-1,4])
meC50_sem = stats.sem(meC50[:,-1,4])
meC70_sem = stats.sem(meC70[:,-1,4])
meC100_sem = stats.sem(meC100[:,-1,4])

In [ ]:
import math
from scipy.optimize import curve_fit

def sigmoid(x):
    sig = 1 / (1 + math.exp(-x))
    return sig

In [ ]:
e_popt, e_pcov = curve_fit(sigmoid, [1,10,20,30,50,70,100], [np.mean(eC1_cum),np.mean(eC10_cum),np.mean(eC20_cum),\
                                        np.mean(eC30_cum),np.mean(eC50_cum),np.mean(eC70_cum),np.mean(eC100_cum)])
me_popt, me_pcov = curve_fit(sigmoid, [1,10,20,30,50,70,100], [np.mean(meC1_cum),np.mean(meC10_cum),\
                    np.mean(meC20_cum),np.mean(meC30_cum),np.mean(meC50_cum),np.mean(meC70_cum),np.mean(meC100_cum)])

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(18,10))
x = [1,10,20,30,50,70,100]
e_yerr = [eC1_sem,eC10_sem,eC20_sem,eC30_sem,eC50_sem,eC70_sem,eC100_sem]
me_yerr = [meC1_sem,meC10_sem,meC20_sem,meC30_sem,meC50_sem,meC70_sem,meC100_sem]
ax.plot(x,[np.mean(eC1_cum),np.mean(eC10_cum),np.mean(eC20_cum),np.mean(eC30_cum),\
                                 np.mean(eC50_cum),np.mean(eC70_cum),np.mean(eC100_cum)],c='cyan',marker='o',mfc='g')
ax.plot(x,[np.mean(meC1_cum),np.mean(meC10_cum),np.mean(meC20_cum),np.mean(meC30_cum),\
                                 np.mean(meC50_cum),np.mean(meC70_cum),np.mean(meC100_cum)],c='orange',marker='o')
ax.errorbar(x,[np.mean(eC1_cum),np.mean(eC10_cum),np.mean(eC20_cum),np.mean(eC30_cum),\
                                 np.mean(eC50_cum),np.mean(eC70_cum),np.mean(eC100_cum)],yerr=e_yerr)
ax.errorbar(x,[np.mean(meC1_cum),np.mean(meC10_cum),np.mean(meC20_cum),np.mean(meC30_cum),\
                                 np.mean(meC50_cum),np.mean(meC70_cum),np.mean(meC100_cum)],yerr=me_yerr)

In [ ]:
stats.f_oneway([np.mean(eC1_cum),np.mean(eC10_cum),np.mean(eC20_cum),np.mean(eC30_cum),\
                                 np.mean(eC50_cum),np.mean(eC70_cum),np.mean(eC100_cum)],[np.mean(meC1_cum),np.mean(meC10_cum),np.mean(meC20_cum),np.mean(meC30_cum),\
                                 np.mean(meC50_cum),np.mean(meC70_cum),np.mean(meC100_cum)])[1]

In [ ]:
for c in ['1','10','20','30','50','70','100']:
    exec(f'avg_LeC{c}_trajectory = np.mean(comb_LeC{c}[:,:,4],axis=0)')
    exec(f'avg_ReC{c}_trajectory = np.mean(comb_ReC{c}[:,:,4],axis=0)')
    exec(f'avg_LeC{c}_SEM = stats.sem(comb_LeC{c}[:,:,4],axis=0)')
    exec(f'avg_ReC{c}_SEM = stats.sem(comb_ReC{c}[:,:,4],axis=0)')
    
    
    exec(f'avg_LmeC{c}_trajectory = np.mean(comb_LmeC{c}[:,:,4],axis=0)')
    exec(f'avg_RmeC{c}_trajectory = np.mean(comb_RmeC{c}[:,:,4],axis=0)')
    exec(f'avg_LmeC{c}_SEM = stats.sem(comb_LmeC{c}[:,:,4],axis=0)')
    exec(f'avg_RmeC{c}_SEM = stats.sem(comb_RmeC{c}[:,:,4],axis=0)')

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,8))
plt.title("Average Change in Cumulative Turning Angle over Time for 100% Contrast",fontsize=15)
ax.set_xlabel("Time",fontsize=13)
ax.set_ylabel("Cumulative Turning Angle (rad)",fontsize=13)
ax.set_xlim(0,3000)
ax.plot(avg_ReC100_trajectory,c="m",label='E2')
ax.plot(avg_RmeC100_trajectory,c="c",label='Methionine')
ax.plot(avg_LeC100_trajectory,c="m")
ax.plot(avg_LmeC100_trajectory,c="c")
plt.legend()
plt.show()

In [ ]:
print('Right flow, contrast 1%, ANOVA p-value:',stats.f_oneway(avg_ReC1_trajectory,avg_RmeC1_trajectory)[1])
print('Right flow, contrast 10%, ANOVA p-value:',stats.f_oneway(avg_ReC10_trajectory,avg_RmeC10_trajectory)[1])
print('Right flow, contrast 20%, ANOVA p-value:',stats.f_oneway(avg_ReC20_trajectory,avg_RmeC20_trajectory)[1])
print('Right flow, contrast 30%, ANOVA p-value:',stats.f_oneway(avg_ReC30_trajectory,avg_RmeC30_trajectory)[1])
print('Right flow, contrast 50%, ANOVA p-value:',stats.f_oneway(avg_ReC50_trajectory,avg_RmeC50_trajectory)[1])
print('Right flow, contrast 70%, ANOVA p-value:',stats.f_oneway(avg_ReC70_trajectory,avg_RmeC70_trajectory)[1])
print('Right flow, contrast 100%, ANOVA p-value:',stats.f_oneway(avg_ReC100_trajectory,avg_RmeC100_trajectory)[1])

In [ ]:
print('Left flow, contrast 1%, ANOVA p-value:',stats.f_oneway(avg_LeC1_trajectory,avg_LmeC1_trajectory)[1])
print('Left flow, contrast 10%, ANOVA p-value:',stats.f_oneway(avg_LeC10_trajectory,avg_LmeC10_trajectory)[1])
print('Left flow, contrast 20%, ANOVA p-value:',stats.f_oneway(avg_LeC20_trajectory,avg_LmeC20_trajectory)[1])
print('Left flow, contrast 30%, ANOVA p-value:',stats.f_oneway(avg_LeC30_trajectory,avg_LmeC30_trajectory)[1])
print('Left flow, contrast 50%, ANOVA p-value:',stats.f_oneway(avg_LeC50_trajectory,avg_LmeC50_trajectory)[1])
print('Left flow, contrast 70%, ANOVA p-value:',stats.f_oneway(avg_LeC70_trajectory,avg_LmeC70_trajectory)[1])
print('Left flow, contrast 100%, ANOVA p-value:',stats.f_oneway(avg_LeC100_trajectory,avg_LmeC100_trajectory)[1])